In [1]:
from smartsim import Experiment
from smartsim.database import SlurmOrchestrator
from smartsim.settings import SrunSettings
from smartsim import slurm

def launch_cluster_orc(experiment, port, alloc):
    """Just spin up a database cluster, check the status
       and tear it down"""

    db = SlurmOrchestrator(port=port,
                            db_nodes=1,
                            batch=False,
                            alloc=alloc,
                            interface="ib0")
    

    # generate directories for output files
    # pass in objects to make dirs for
    experiment.generate(db, overwrite=True)

    # start the database on interactive allocation
    experiment.start(db, block=True)

    return db

def create_uploader(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="data_uploader.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    uploader = experiment.create_ensemble("uploader", replicas=2, run_settings=srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    uploader.attach_generator_files(to_copy=["./tf/data_uploader.py"])
    experiment.generate(uploader, overwrite=True)
    return uploader


def create_trainer(experiment, alloc):

    srun = SrunSettings(exe="python",
                        exe_args="training_service.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_tasks(1)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./tf/training_service.py")
    experiment.generate(trainer, overwrite=True)
    return trainer


def create_trainer_hvd(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="training_service_hvd.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./tf/training_service_hvd.py")
    experiment.generate(trainer, overwrite=True)
    return trainer

In [2]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

10:11:37 osprey.us.cray.com SmartSim[8627] INFO Allocation successful with Job ID: 528400


In [11]:
exp = Experiment("launch_streaming", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 2)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer(exp, alloc)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

10:34:54 osprey.us.cray.com SmartSim[8627] INFO Working in previously created experiment
10:35:27 osprey.us.cray.com SmartSim[8627] INFO Working in previously created experiment
10:35:34 osprey.us.cray.com SmartSim[8627] INFO Working in previously created experiment
10:35:42 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.17): New
10:35:42 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.18): New
10:35:42 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.19): New
10:35:47 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.17): Running
10:35:47 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.18): Running
10:35:47 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.19): Running
10:35:52 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.17): Running
10:35:52 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.18): Running
10:35:52 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.19): Running
10:35:57 osprey.us.cray.com SmartSim[8627] 

KeyboardInterrupt: 

In [14]:
exp.stop(db, uploader_model, trainer_model)

10:53:46 osprey.us.cray.com SmartSim[8627] INFO Stopping model uploader_0 with job name uploader_0-CG4Z07I2SIUF
10:53:46 osprey.us.cray.com SmartSim[8627] INFO Stopping model uploader_1 with job name uploader_1-CG4Z07I2TI0S
10:53:46 osprey.us.cray.com SmartSim[8627] INFO Stopping model orchestrator_0 with job name orchestrator_0-CG4YZS5KB8Z8


In [15]:
exp = Experiment("launch_streaming_hvd", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 8)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer_hvd(exp, alloc, 1, 8)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

10:54:36 osprey.us.cray.com SmartSim[8627] INFO Working in previously created experiment
10:55:09 osprey.us.cray.com SmartSim[8627] INFO Working in previously created experiment
10:55:16 osprey.us.cray.com SmartSim[8627] INFO Working in previously created experiment
10:55:24 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.25): New
10:55:24 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.26): New
10:55:24 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.27): New
10:55:29 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.25): Running
10:55:29 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.26): Running
10:55:29 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.27): Running
10:55:34 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.25): Running
10:55:34 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.26): Running
10:55:34 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.27): Running
10:55:39 osprey.us.cray.com SmartSim[8627] 

KeyboardInterrupt: 

In [16]:
slurm.release_allocation(alloc)

11:00:08 osprey.us.cray.com SmartSim[8627] INFO Releasing allocation: 528400
11:00:08 osprey.us.cray.com SmartSim[8627] INFO Successfully freed allocation 528400
11:00:11 osprey.us.cray.com SmartSim[8627] INFO uploader_0(528400.25): Cancelled
11:00:11 osprey.us.cray.com SmartSim[8627] INFO uploader_1(528400.26): Cancelled
11:00:11 osprey.us.cray.com SmartSim[8627] INFO trainer(528400.27): Cancelled
11:00:11 osprey.us.cray.com SmartSim[8627] INFO orchestrator_0(528400.24): Cancelled
